In [14]:
# ✅ Standard Libraries
import os
import time
from uuid import uuid4
from typing import Literal, Optional,List
from typing_extensions import TypedDict

# ✅ Third-Party Utilities
import numpy as np
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

# ✅ LangChain Core
from langchain_core.tools import tool
from langchain_core.messages import convert_to_messages
from langchain_core.documents import Document
from langchain_core.runnables import RunnableConfig,RunnableLambda

# ✅ LangChain Models
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_vertexai import ChatVertexAI

# ✅ LangChain MongoDB Integration
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_mongodb.retrievers.hybrid_search import MongoDBAtlasHybridSearchRetriever

# ✅ LangChain Tools / Plugins
from langchain_tavily import TavilySearch
from langchain.output_parsers.openai_functions import (
    JsonOutputFunctionsParser,
    PydanticOutputFunctionsParser
)
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.prompts import ChatPromptTemplate, PromptTemplate

# ✅ LangGraph
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from langchain_community.callbacks import get_openai_callback

# ✅ Pydantic for Schema
from pydantic import BaseModel, Field

In [15]:
# ✅ Load variables from .env file
load_dotenv()

# ✅ Access API keys and URIs
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# ✅ Raise error if any key is missing
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in .env file")
if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY is not set in .env file")

# ✅ Optional: Set as environment variables for downstream tools
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [ ]:
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model
from typing import Optional, Literal
from pydantic import BaseModel, Field

class StateSchema(BaseModel):
    
    id: str = Field(..., description="The name of the product to classify.")
    query: str = Field(..., description="The name of the product to classify.")

    # Prediction outputs
    answer: str = Field(..., description="Choose the best answer from the multiple choices")
    reason: str = Field(..., description="A paragraph summarizing that leading to your conclusion")
    
    # Confidence score of answer
    confidence_score: Optional[float]

    # Contexts from tools
    web_search_flag: Optional[str]

    # Tracking who should handle next
    route_to: Optional[str]

supervisor_prompt = """
You are a supervisor agent responsible for overseeing the product classification funnel and routing logic.

<Role>
- Receive outputs from specialized agents (classification_chooser, checker, funnel).
- Interpret the overall result and produce a final structured summary.
- Decide whether classification is complete, needs rerouting, or should be escalated to a human expert.
- Ensure fallback routing (e.g., to funnel agent or human expert) follows a strict logic path.

<Logic>
The classification workflow proceeds through the following steps:

1. **Initial Classification (classification_chooser_agent):**
   - Predicts taxonomy fields: division, category, segment, class.
   - Also predicts brand, supplier, and a confidence score.
   - Returns reasoning and evidence from:
     - Web search (`web_search_context`)
     - Retrieved product examples (`retrieved_context`)

2. **Verification (checker_agent):**
   - Verifies the semantic and contextual accuracy of the classification.
   - Accepts or rejects it and may adjust the confidence score.

3. **Taxonomy Reuse from Retrieved Products (if applicable):**
   - If `retrieved_context` contains product(s) with highly similar names to the input (e.g., shared prefix/suffix, differing by 1–2 characters),
   - AND those retrieved products include complete taxonomy (i.e., division, category, segment, and class are present and not "OTHERS"),
   → Then:
     - Reuse that taxonomy for the current product.
     - Skip rerouting to funnel agent or relying on web search context.
     - Explain in the reason that taxonomy was reused from retrieved products due to strong name pattern match and valid taxonomy structure.

4. **Mandatory Funnel Taxonomy Check (Always Applied if not using reused taxonomy):**
   - If all taxonomy fields are present but have not yet been validated for structural consistency,
   → Route to the **funnel agent** to ensure division, category, segment, and class are correctly aligned based on internal taxonomy rules.

5. **Funnel Retry (if not yet attempted):**
   - If confidence is low OR any taxonomy fields are missing,
   - And the funnel agent has not yet been tried,
   → Route to the funnel agent for re-classification or completion.

6. **Early Stop – Ambiguous Input Escalation:**
   - If BOTH:
     - Product name is ambiguous or code-like (i.e., lacks semantic context),
     - AND both `web_search_context` and `retrieved_context` lack helpful classification evidence,
   → Then:
     - Set `send_to_expert = true`
     - Indicate that classification cannot proceed due to lack of usable information.

7. **Final Escalation to Human Expert:**
   - If the funnel agent has already been used,
   - And taxonomy remains incomplete, inconsistent, or still mapped to "OTHERS",
   → Escalate to human by setting `send_to_expert = true`.

<Formatting Rules>
- Always output the following fields in **ALL UPPERCASE**:
  - `division`
  - `category`
  - `segment`
  - `class`
- Preserve original casing for `brand` and `supplier`
- Do not abbreviate, truncate, or invent taxonomy terms
- Provide a clear `reason` explaining the final routing or decision
"""





# === CREATE AND COMPILE SUPERVISOR ===
supervisor = create_supervisor(
    model=llm,
    #state_schema=StateSchema,
    agents=[classification_chooser_agent, checker_agent, funnel_agent],
    prompt=supervisor_prompt,
    response_format=SupervisorOutput,
    add_handoff_back_messages=True,
    supervisor_name="supervisor",
).compile()